In [1]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/data/labelled_data.csv')
# pd.set_option('display.max_columns', None)
# df.head()

In [3]:
# print(df.info())
# print()
# print(df.shape)
# print()
# df.describe()
# print()
# for i in df.columns:
#     print(i,'---->',df[i].nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17280 entries, 0 to 17279
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               17280 non-null  int64  
 1   timestamps               17280 non-null  object 
 2   BMS_state                17279 non-null  float64
 3   BMS_soc                  17279 non-null  float64
 4   BMS_soh                  17279 non-null  float64
 5   BMS_bus_voltage          17279 non-null  float64
 6   BMS_bus_current          17279 non-null  float64
 7   BMS_isolation            17279 non-null  float64
 8   BMS_max_cell_temp        17279 non-null  float64
 9   BMS_max_cell_temp_id     17279 non-null  float64
 10  BMS_min_cell_temp        17279 non-null  float64
 11  BMS_min_cell_temp_id     17279 non-null  float64
 12  BMS_max_cell_voltage     17279 non-null  float64
 13  BMS_max_cell_voltage_id  17279 non-null  float64
 14  BMS_min_cell_voltage  

In [4]:
from pycaret.classification import *
from imblearn.over_sampling import RandomOverSampler


ignore = ['Unnamed: 0','Anomaly_Score','timestamps']

# Initialize the setup
clf1 = setup(data=df,
             target='Anomaly',
             session_id=123,
             train_size=0.7,
             preprocess=True,
             imputation_type='simple',
             categorical_imputation='mode',
             numeric_imputation='mean',
             normalize=True,
             ignore_features=ignore,
             normalize_method='robust',
             combine_rare_levels=True,
             rare_level_threshold=0.1,
             data_split_stratify=True,
             fold_strategy='stratifiedkfold',
             fold=3,
             fold_shuffle=True,
             n_jobs=None,
             silent=True,
             fix_imbalance=True,
             fix_imbalance_method=RandomOverSampler(),
             ignore_low_variance=True,
             handle_unknown_categorical=True,
             unknown_categorical_method='least_frequent',
             feature_selection=True,
             feature_selection_method= "classic",
             )

,Description,Value
0,session_id,123
1,Target,Anomaly
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(17280, 34)"
5,Missing Values,True
6,Numeric Features,20
7,Categorical Features,10
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
X_train = get_config('X_test')
X_test = get_config('X_test')

X_train.to_csv("D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/data/X_train.csv")
X_test.to_csv("D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/data/X_test.csv")

In [ ]:
models()

In [ ]:
# Compare models to find the best one
compare_models()

In [ ]:
ada = create_model('ada')

In [ ]:
tuned_ada = tune_model(ada)

In [ ]:
plot_model(tuned_ada)

In [ ]:
evaluate_model(tuned_ada)

In [ ]:
pred_holdout = predict_model(tuned_ada)

In [ ]:
from pycaret.classification import get_leaderboard
leaderboard = get_leaderboard()

In [ ]:
# Tune the Adaboost Classifier model
# import warnings
# warnings.filterwarnings('ignore')
tuned_ada_model = tune_model(best_model,n_iter=3,optimize='Accuracy',verbose=True)

In [ ]:
# Access transformed training data
transformed_train_data = get_config('X_train')
transformed_test_data = get_config('X_test')

In [ ]:
train_predictions = tuned_ada.predict(transformed_train_data)
test_predictions = tuned_ada.predict(transformed_test_data)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
print(round(accuracy_score(get_config('y_train'),train_predictions),4))
print(round(accuracy_score(get_config('y_test'),test_predictions),4))


In [ ]:
predict_model(tuned_ada)

In [ ]:
import mlflow

mlflow.set_experiment(experiment_name='training_experiment_1')
mlflow.end_run()

with mlflow.start_run():
    clf1 = setup(data=df,target='Anomaly',session_id=123,train_size=0.7,
             preprocess=True,
             imputation_type='simple',
             categorical_imputation='mode',
             numeric_imputation='mean',
             normalize=True,
             ignore_features=ignore,
             normalize_method='robust',
             combine_rare_levels=True,
             rare_level_threshold=0.1,
             data_split_stratify=True,
             fold_strategy='stratifiedkfold',
             fold=3,
             fold_shuffle=True,
             n_jobs=None,
             silent=True,fix_imbalance=True,
             fix_imbalance_method=RandomOverSampler(),
             ignore_low_variance=True,
             handle_unknown_categorical=True,
             unknown_categorical_method='least_frequent',
             feature_selection=True,
             feature_selection_method= "classic",
             )
    best = compare_models(include=['lightgbm','ada','rf'])

In [ ]:
! mlflow ui